In [1]:
import sys;sys.path.append('d:/code/pywfn')

# 空间性质

计算分子空间中某些点的性质，全都是空间函数`f(x,y,z)`

所有的空间性质的计算器都包含在pywfn.gridprop子包下，其中每一个模块封装了一种类型的空间性质计算器

其包含的模块有

- `wfnfunc` 分子轨道波函数
- `density` 与电子密度相关的函数
- `potential` 静电势相关函数
- `hmowfn` 休克尔分子轨道波函数
- `dftgrid` dft计算的空间格点

每个模块下都有一个Calculator类，实例化时传入要计算的分子即可

## 波函数

### 分子轨道波函数

计算指定`空间格点`指定`分子轨道`的分子轨道波函数，返回二维矩阵，形状为[nobt,npos]（分子轨道数量，格点数量）

**示例代码**

下面代码是计算苯分子第20和第21个分子轨道的20个随机点的波函数

In [2]:
from pywfn.base import Mole
from pywfn.reader import LogReader
from pywfn.gridprop import wfnfunc
import numpy as np

mol=Mole(LogReader(rf"./mols/C6H6.out"))
caler=wfnfunc.Calculator(mol)

grids=np.random.rand(20,3)
vals=caler.obtWfns(grids,obts=[19,20])
print(vals)
print(vals.shape)

[[0.0021 0.0089 0.0267 0.0294 0.0002 0.0136 0.0437 0.0172 0.006  0.0028
  0.0009 0.0479 0.0054 0.0032 0.0281 0.0345 0.0185 0.0477 0.0161 0.0338]
 [0.0244 0.0073 0.006  0.0237 0.0055 0.015  0.034  0.0029 0.0019 0.0049
  0.0013 0.0497 0.0017 0.0161 0.0205 0.0048 0.0014 0.0146 0.0209 0.0331]]
(2, 20)


### 原子轨道波函数
计算指定空间格点的所有原子轨道的波函数

**示例代码**

下列代码计算苯分子空间中随机的10个点处102个原子轨道的波函数、波函数一阶导数和二阶导数

In [3]:
from pywfn.base import Mole
from pywfn.reader import LogReader
from pywfn.gridprop import wfnfunc
import numpy as np

mol=Mole(LogReader(rf"./mols/C6H6.out"))
caler=wfnfunc.Calculator(mol)
grids=np.random.rand(10,3)
wfns0,wfns1,wfns2=caler.atoWfns(grids,level=2)
print(wfns0.shape)
print(wfns1.shape)
print(wfns2.shape)

(102, 10)
(102, 10, 3)
(102, 10, 3, 3)


## 电子密度

### 分子总电子密度

计算空间中某些点处分子的总电子密度及电子密度的一阶导数和二阶导数

**示例代码**

In [4]:
from pywfn.base import Mole
from pywfn.reader import LogReader
from pywfn.gridprop import density
import numpy as np

mol=Mole(LogReader(rf"./mols/C6H6.out"))
caler=density.Calculator(mol)

grids=np.random.rand(2,3)
dens0,dens1,dens2=vals=caler.molDens(grids,level=2)
print(dens0)
print(dens1)
print(dens2)
print(dens0.shape)
print(dens1.shape)
print(dens2.shape)

[0.0324 0.0814]
[[ 0.034   0.0409 -0.0118]
 [ 0.1288  0.0544 -0.017 ]]
[[[ 0.1033  0.0141 -0.0161]
  [ 0.0141  0.1118 -0.0195]
  [-0.0161 -0.0195 -0.0242]]

 [[ 0.2033  0.0184 -0.0339]
  [ 0.0184  0.1539 -0.0139]
  [-0.0339 -0.0139 -0.0879]]]
(2,)
(2, 3)
(2, 3, 3)


### 预分子电子密度
将分子中原子的自由电子密度加和即为预分子电子密度，无需量化计算，只需要知道几何结构即可

**示例代码**

In [5]:
from pywfn.base import Mole
from pywfn.reader import LogReader
from pywfn.gridprop import density
import numpy as np

mol=Mole(LogReader(rf"./mols/C6H6.out"))
caler=density.Calculator(mol)
grids=np.random.rand(2,3)
caler.proMolDens(grids)

array([0.0425, 0.0434])

### RDG函数
约化密度梯度：Reduced Density Gradient

用来识别弱相互作用

**计算公式**

$$
s=\frac{1}{2(3\pi ^2)^{1/3}}\frac{|\Delta \rho|}{\rho ^{4/3}}
$$

**示例代码**

In [6]:
from pywfn.base import Mole
from pywfn.reader import LogReader
from pywfn.gridprop import density
import numpy as np

mol=Mole(LogReader(rf"./mols/C6H6.out"))
caler=density.Calculator(mol)
grids=np.random.rand(2,3)
caler.RDG(grids)

array([0.6766, 0.9173])

### IRI函数

在RDG上改良得到，能够同时展示化学键（如共价键、离子键）和弱相互作用（如氢键、π-π堆积）的区域

**计算公式**

$$
s=\frac{|\Delta \rho|}{\rho ^\alpha}
$$

其中α是可调节参数，默认值为1.1

**示例代码**

In [7]:
from pywfn.base import Mole
from pywfn.reader import LogReader
from pywfn.gridprop import density
import numpy as np

mol=Mole(LogReader(rf"./mols/C6H6.out"))
caler=density.Calculator(mol)
grids=np.random.rand(2,3)
caler.IRI(grids)

array([2.3928, 2.3151])